In [9]:
source("http://www.bioconductor.org/biocLite.R")
biocLite("affy")
library(affy)

# Don't have to set this if Jupyter notebook in `src` directory
#setwd("/Users/dominicanene/Google Drive/Northwestern/2018-2019/q2/bioinformatics/project/src")

# directory that holds raw files
rawdata.dir <- "../data/raw"
dir.alzheimer <- file.path(rawdata.dir,"AH")
dir.cte <- file.path(rawdata.dir,"CTE")

# directory that holds clean/processed files
cleandata.dir <- "../data/clean"



Bioconductor version 3.7 (BiocInstaller 1.30.0), ?biocLite for help
A newer version of Bioconductor is available for this version of R,
  ?BiocUpgrade for help
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.7 (BiocInstaller 1.30.0), R 3.5.1 (2018-07-02).
Installing package(s) ‘affy’
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'caTools', 'dplyr', 'evaluate', 'forcats', 'forge', 'formatR',
  'geometry', 'gower', 'haven', 'knitr', 'later', 'lava', 'lme4', 'maptools',
  'Matrix', 'mgcv', 'modelr', 'mongolite', 'openssl', 'purrr', 'R6',
  'RcppArmadillo', 'RCurl', 'readxl', 'repr', 'slam', 'sparklyr', 'stringi',
  'stringr', 'sys', 'tidyr', 'xfun', 'XML', 'zip'


# alzheimer Dataset 
*"Temporal relationships between cognitive aging and molecular changes in a critical brain region."* <br> <br>
*link to paper ->* [AH Paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2661568/) <br>
*link to data ->* [AH data](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE9990) <br>



 *1 microarray per an animal* <br>
 *49 microarrays*

| AH Data       |group1|group2|group3|group4|group5|         
| ------------- |------|------|------|------|------|   
| Number Rats   |  9   |    9 |  9   |  9   | 13   |
| Age (Months)  |  3   | 6    |  9   |  12  | 23   |

 
*Genes that changed with aging were assigned to one-of-four onset-age ranges based upon template. Evidence of cognitive impairment appeared in the 12-month-old group (midlife) and was increased further in the 23-month-old group.*

| onset-age classes |early|intermediate|midlife| late | 
|-------------------|-----|------------|-------|------|
| onset-age range   |3-6  |6-9         |9-13   | 12-23|     
> So standard differential expression analysis between $M_t$ and $M_{t-1}$ to identify genes that change with aging? 

<br>

**Cleaning method of authors**
1.  Unique, annotated gene probe sets rated present on at least 6 chips in the study were retained for statistical analysis
    - The presence call cutoff of 6 was chosen based on factorial analysis indicating that there was a < 5% probability that 6 presence calls would be found in a set of 49 chips by chance. 
    > Cool! we'll just take this for granted :)
2. Outlier signal intensity (expression) data (> 2 SD from the group mean for each probe set in each age group) were treated as missing values
> This is odd. Need to review rationale. 
3.  Genes that are left and differentially expressed between times t and t-1 (using False Discovery Rate) were designated "Aging-Related Genes"

**Below are maps between GEO ID Codes for microarray data sets and mneumonic names for the microarray datasets. For example `GSM252558` refers to the microarray that represents $9^{th}$ rat in the 23month age group.** 

In [10]:
ah.names.23Months = list("GSM252558" = "23Months_9",
                     "GSM252557" = "23Months_8",
                     "GSM252556" = "23Months_7", 
                     "GSM252555" = "23Months_6", 
                     "GSM252554" = "23Months_5", 
                     "GSM252553" = "23Months_4", 
                     "GSM252552" = "23Months_3", 
                     "GSM252551" = "23Months_15", 
                     "GSM252550" = "23Months_14", 
                     "GSM252549" = "23Months_13", 
                     "GSM252548" = "23Months_11", 
                     "GSM252547" = "23Months_10", 
                     "GSM252546" = "23Months_1")

ah.names.12Months = list("GSM252545" = "12Months_9",
                      "GSM252544" = "12Months_8",
                      "GSM252543" = "12Months_7", 
                      "GSM252542" = "12Months_6", 
                      "GSM252541" = "12Months_5", 
                      "GSM252540" = "12Months_4", 
                      "GSM252539" = "12Months_3", 
                      "GSM252538" = "12Months_2", 
                      "GSM252537" = "12Months_1")

ah.names.9Months = list("GSM252536" = "9Months_9",
                      "GSM252535" = "9Months_8",
                      "GSM252534" = "9Months_7", 
                      "GSM252533" = "9Months_6", 
                      "GSM252532" = "9Months_5", 
                      "GSM252531" = "9Months_4", 
                      "GSM252530" = "9Months_3", 
                      "GSM252529" = "9Months_2", 
                      "GSM252528" = "9Months_1")

ah.names.6Months = list("GSM252527" = "6Months_9",
                      "GSM252526" = "6Months_8",
                      "GSM252525" = "6Months_7", 
                      "GSM252524" = "6Months_6", 
                      "GSM252523" = "6Months_5", 
                      "GSM252522" = "6Months_4", 
                      "GSM252521" = "6Months_3", 
                      "GSM252520" = "6Months_2", 
                      "GSM252519" = "6Months_1")

ah.names.3Months = list("GSM252518" = "3Months_9",
                      "GSM252517" = "3Months_8",
                      "GSM252516" = "3Months_7", 
                      "GSM252515" = "3Months_6", 
                      "GSM252514" = "3Months_5", 
                      "GSM252513" = "3Months_4", 
                      "GSM252512" = "3Months_3", 
                      "GSM252511" = "3Months_2", 
                      "GSM252510" = "3Months_1")

# cte Dataset 
***Goal of study***: most of functional deficit from brain trauma are delayed. Goal is to identify potential molecular mechanisms underlying such delayed responses by compared gene expression patterns  at 4, 8, 24, and 72h after moderate levels of lateral fluid percussion–induced brain injury in rats

***Relevance*** The condition known as mild traumatic brain injury is more commonly referred to by the term concussion. While a severe concussion will normally be referred to as a traumatic brain injury or TBI, normal concussions are referred to as being mild traumatic brain injuries (MTBI) due to the fact that a single injury of this type will not typically cause any serious long term health consequences. Several repeated mild traumatic brain injuries, however, may lead to the life-changing and potentially debilitating condition known as chronic traumatic encephalopathy (CTE). [REFERNCE](http://www.protectthebrain.org/Brain-Injury-Research/Mild-Traumatic-Brain-Injuries-MTBI-.aspx) <br> <br>
***Note***: *CTE data set says the injured is "moderate traumatic," this might be too severe, but I've checked and this seems to be the best dataset. So it worth adding a line about this in our final report.* <br>
***Note***: *The paper and the abstract say that the longest duration of measurement is 72h, but actually the longest duration of the data is [21 days](https://www.ncbi.nlm.nih.gov/sites/GDSbrowser?acc=GDS2851)!*  Also the paper says that there are 4 time points, but actually there are 6 time periods in the data! <br>
***Problem:*** *What are the ages of CTE rats? This could be important since AH dataset defined wholly by age and we are comparing the two*

***The Data***
- Injured rats: At each time point 3 rats were injured (4h, 8h, 24h, and 72h)
- Sham rats: *subjected to surgery but not injured*
    - four rats at the 4h time point 
    - two rats at {8h, 24h, and 72h}
- Naive rats: 3 aged matched rats present throughout the study and unharmed. 

At predetermined times after surgery (4, 8, 24, and 72 h) sham and injured were decapitated. 

***Filtering & Analysis***
- presence cutoff being probe sets that were detected in 40% or more of the microarrays
> This cutoff more stringent then AH dataset. AH had cutoff of atleast 6 chips, hence probe sets detected in atleast 12% of micro arrays included.  
> ***We will need a rational for whatever method we choose!***

- Normalization: For each probe set, signal
intensity from injured and sham controls were normalized to the mean signal intensity generated from the same
cortical region from three naive rats
> So use the naive rats to control for the effects of surgery. For our purposes we only focus on injured dataset to match the analysis in AH (since AH has no control). Do this by shifting injured datasets to be centered around naive rats means. <br> ¿Then normalize and get expressions using rma().?


*link to paper ->* [CTE Paper](https://www.researchgate.net/profile/Ibolja_Cernak/publication/9032663_Gene_Expression_Profile_Changes_Are_Commonly_Modulated_across_Models_and_Species_after_Traumatic_Brain_Injury/links/09e415016e266e9c68000000/Gene-Expression-Profile-Changes-Are-Commonly-Modulated-across-Models-and-Species-after-Traumatic-Brain-Injury.pdf?origin=publication_detail) <br>
*link to data ->* [CTE data: Splash page](https://www.ncbi.nlm.nih.gov/sites/GDSbrowser?acc=GDS2851) <br>
*link to data ->* [CTE data: Description](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE2392) <br>
*link to data ->* [CTE data: Actual Download page](https://www.ncbi.nlm.nih.gov/geo/download/?acc=GDS2851) <br>

**Below are maps between GEO ID Codes for microarray data sets and mneumonic names for the microarray datasets. For example `GSM44667` refers to the microarray that represents the rat identified by 2bA-s2, who is injured, and measured 4h after injury, `JMR-RatInj4h-2bA-s2`**

In [11]:
cte.names.30m = list("GSM44533" = "JMR-RatInj30m-5aA-s2",
                     "GSM44500" =  "JMR-RatInj30m-1aA-s2",
                     "GSM44492" = "JMR-RatInj30m-3aA-s2",
                     "GSM44480" = "JMR-RatInj30m-4aA-s2",
                     "GSM44473" = "JMR-RatInj30m-2aA-s2")

cte.names.4h = list("GSM44667" = "JMR-RatInj4h-2bA-s2",
                    "GSM44498" = "JMR-RatInj4h-3bA-s2",
                    "GSM44466" = "JMR-RatInj4h-1bA-s2")
                    
cte.names.8h = list("GSM44532" = "JMR-RatInj8h-1aA-s2", 
                    "GSM44531" = "JMR-RatInj8h-2aA-s2",
                    "GSM44491" = "JMR-RatInj8h-3aA-s2")
                                        
cte.names.24h = list("GSM44493" = "JMR-RatInj24hf-2aA-s2",
                     "GSM44482" = "JMR-RatInj24hf-3aA-s2",
                     "GSM44477" =  "JMR-RatInj24hf-1aA-s2")
                                         
cte.names.72h = list("GSM44549" = "JMR-RatInj72h-3aA-s2",
                     "GSM44520" = "JMR-RatInj72h-1aA-s2",
                     "GSM44484" = "JMR-RatInj72h-2aA-s2")
                            
cte.names.21d = list("GSM44497" = "JMR-RatInj21d-1bA-s2",
                     "GSM44481" = "JMR-RatInj21d-2aA-s2", 
                     "GSM44471" = "JMR-RatInj21d-3aA-s2")

cte.naive.names = list("GSM44513" = "JMR-RatNaive-1cA-s2",  
                       "GSM44496" = "JMR-RatNaive-2cA-s2",
                       "GSM44478" = "JMR-RatNaive-3bA-s2")

#To be complete include names below but the sham dataset will not be used in analyis
cte.sham.names.30m = list("GSM44509" = "JMR-RatSham30m-3aA-s2",
                          "GSM44494"= "JMR-RatSham30m-1aA-s2",
                          "GSM44489" = "JMR-RatSham30m-2aA-s2",
                            "GSM44488" = "JMR-RatSham30m-4aA-s2")
    
cte.sham.names.4h = list("GSM44529" = "JMR-RatSham4h-3aA-s2",
                         "GSM44528" = "JMR-RatSham4h-4aA-s2",
                          "GSM44511" = "JMR-RatSham4h-1aA-s2", 
                         "GSM44486" = "JMR-RatSham4h-2aA-s2")
                         
cte.sham.names.8h = list("GSM44530" = "JMR-RatSham8h-1aA-s2",
                         "GSM44467" = "JMR-RatSham8h-2aA-s2",
                         "GSM44467" = "JMR-RatSham8h-2aA-s2")
                         
cte.sham.names.24h = list("GSM44508" = "JMR-RatSham24hf-2aA-s2",
                          "GSM44490" = "JMR-RatSham24hf-1aA-s2",
                          "GSM44490" = "JMR-RatSham24hf-1aA-s2")
                          
cte.sham.names.72h = list("GSM44485" = "JMR-RatSham72h-2aA-s2", 
                          "GSM44483" = "JMR-RatSham72h-1aA-s2")
                          
cte.sham.names.21d = list("GSM44507" = "JMR-Sham21d-1aA-s2",
                          "GSM44495" = "JMR-Sham21d-2aA-s2")

In [13]:
library(purrr)

#column names: Convert into a named vector
ah.colnames <- c(unlist(ah.names.3Months),
                 unlist(ah.names.6Months),
                 unlist(ah.names.9Months),
                 unlist(ah.names.12Months),
                 unlist(ah.names.23Months))

cte.colnames <- c(unlist(cte.names.30m),
                 unlist(cte.names.4h),
                 unlist(cte.names.8h),
                 unlist(cte.names.24h),
                 unlist(cte.names.72h),
                 unlist(cte.names.21d))

cte.naive.colnames<-unlist(cte.naive.names)

#Get raw CEL files names
fnames.ah <- map_chr(names(ah.colnames), (function(fn) (paste(fn,".CEL.gz",sep=""))))
fnames.cte <- map_chr(names(cte.colnames), (function(fn) (paste(fn,".CEL.gz",sep=""))))
fnames.naive.cte <- map_chr(names(cte.naive.colnames), (function(fn) (paste(fn,".CEL.gz",sep=""))))

#Load Affymetrix data                                                         
pcraw.ah<- ReadAffy(filenames=fnames.ah,celfile.path=dir.alzheimer)
pcraw.cte<- ReadAffy(filenames=fnames.cte,celfile.path=dir.cte)
pcraw.naive.cte<-ReadAffy(filenames=fnames.naive.cte,celfile.path=dir.cte)                                               

sampleNames(pcraw.ah) <-ah.colnames
sampleNames(pcraw.cte) <-cte.colnames
sampleNames(pcraw.naive.cte) <-cte.naive.colnames

### Filtering the genes
*Both AH & CTE have "presence call cutoff" filter*  <br>
**AH paper**
> Unique, annotated gene probe sets rated present on at least 6 chips in the study were retained for statistical analysis. (pg 4, section: Statistics) 
 
**CTE paper**
> We based all further analysis only on those probe sets that were detected in 40% or more of the microarrays comprising the complete microarray series for each experimental model (pg 5, section: Data Filtering and Statistical Analysis) 


The term "presence call" is from lecture 2 (slide 6), to calculate we create the discriminant score $$R = \frac{PM-MM}{PM+MM}$$
When $R = 0$ the gene is not present, and so we want to test the hypothesis:
\begin{align*}&H_0: E(R) = 0 \\ &H_a: E(R) \neq 0 .\end{align*} To do this we apply the Wilcoxson's signed rank test for each probe set. Then we keep the genes that are detected in at least 40% of the microarrays. The test will at the 95% confidence level. <br> <br>

**Procedure**
1. Keep genes in at least 40% of microarrays using Wilcoxon signed rank-based gene expression presence/absence detection algorithm. `mas5calls`
1. Normalize
    - AH: log transform and then quantile normalization
    - CTE: log transform, subtract naive means, and then quantile normalization
2. Get expressions using Robust Multi-Array (RMA)


**Cleaning AH**

In [25]:
# Present, Absent, Marginal Intensity Measurment
ah.pma<- mas5calls(pcraw.ah)
ah.pma<- exprs(ah.pma) # "P", "M" or "A" representing a call of present, marginal or absent;
ah.pma = ah.pma == "P"| ah.pma == "M" # Boolean matrix: 1 = Present or Marginal, 0 = Absent
ah.pma = rowSums(ah.pma) # Get total number of Present or Marginal for a gene
ah.pma.Idx = which(ah.pma >= length(ah.colnames)*.5) #Get indices of genes present in >= 40% of arrays

#Robust Micro Array Avg w/ Quantile Normalization and log scaling. 
ah.ExprDta <-rma(pcraw.ah)
alzheimer.dta<-exprs(ah.ExprDta)
alzheimer.dta <- alzheimer.dta[ah.pma.Idx,] #filter for sig genes. 

#Save data
write.table(alzheimer.dta, file = file.path(cleandata.dir,"alzheimer.txt"), sep = "\t", row.names = TRUE,col.names =TRUE)


Getting probe level data...
Computing p-values
Making P/M/A Calls
Background correcting
Normalizing
Calculating Expression


**Cleaning CTE**

In [24]:
# Present, Absent, Marginal Intensity Measurment
cte.pma<- mas5calls(pcraw.cte)
cte.pma<- exprs(cte.pma) 
cte.pma = cte.pma == "P"| cte.pma == "M" 
cte.pma = rowSums(cte.pma) 
cte.pma.Idx = which(cte.pma >= length(cte.colnames)*.5) 

# Subtract the means of naive 
exprs(pcraw.cte)<-exprs(pcraw.cte) - rowMeans(exprs(pcraw.naive.cte)) #RMA uses PM, but demeaning the actual expressions to be safe
pm(pcraw.cte)<-pm(pcraw.cte) - rowMeans(pm(pcraw.naive.cte)) 
mm(pcraw.cte)<-mm(pcraw.cte) - rowMeans(mm(pcraw.naive.cte)) 

#Robust Micro Array Avg w/ Quantile Normalization and log scaling. 
cte.ExprDta <-rma(pcraw.cte)
cte.dta<-exprs(cte.ExprDta)
cte.dta <- cte.dta[cte.pma.Idx,] 

#Save data
write.table(cte.dta, file = file.path(cleandata.dir,"cte.txt"), sep = "\t", row.names = TRUE,col.names =TRUE)

Getting probe level data...
Computing p-values
Making P/M/A Calls
Background correcting
Normalizing
Calculating Expression


### Things to possibly add/change
 - 40% intensity call cutoff -> need rational
 - Subtracting mean of naive rats from cte -> correct? 
 - Add -> Outlier signal intensity (expression) data (> 2 SD from the group mean for each probe set in each age group) were treated as missing values. **This is from AH Data description section**
 - Might want to add graphs like from hw2? to see effects of normalization and demeaning
 
One odd things... if you subtract naive means in CTE (`CTE:line 2-4`) after you filter out sign genes (`CTE: line 6-11`) then the significant genes for CTE is way down compared to AH! After seeing this I switched the order but need rational of why this happens. 

In [26]:
print(paste("Num of sig genes for CTE",length(cte.pma.Idx)))
print(paste("Num of sig genes for AH",length(ah.pma.Idx)))

[1] "Num of sig genes for CTE 6898"
[1] "Num of sig genes for AH 10963"


Need a way to get the number of genes to be managable.